In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Connect to the database and read the Dividend table
engine = create_engine('mysql+pymysql://root:@localhost:3306/stock')
const = engine.connect()
dividend_table = pd.read_sql("SELECT * FROM dividend", const)
dividend_table.head()

,NAME,Q4,Q3,Q2,Q1,DIVIDEND,PRICE,PERCENT,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,WHART,0.1265,0.259,0.193,0.1915,0.7700,0.0,0.0,20000.0,2024-03-08,2024-03-24,,2.0
1,CPNREIT,0.0607,0.257,0.290,0.2470,0.8547,0.0,0.0,60000.0,2024-02-28,2024-03-16,,2.0
2,MCS,0.2000,0.000,0.000,0.0000,0.2000,0.0,0.0,78000.0,2024-03-17,2024-05-08,,2.0
3,AIMIRT,0.2230,0.223,0.223,0.2230,0.8920,0.0,0.0,10000.0,2024-03-15,2024-03-30,,2.0
4,SCC,2.0000,0.000,2.500,0.0000,4.5000,0.0,0.0,600.0,2024-04-07,2024-04-25,,2.0


In [3]:
dividend_table.columns

Index(['NAME', 'Q4', 'Q3', 'Q2', 'Q1', 'DIVIDEND', 'PRICE', 'PERCENT',
       'SHARES', 'XDATE', 'PAIDDATE', 'KIND', 'ACTUAL'],
      dtype='object')

In [4]:
# Read the excel file and convert xdate & paiddate columns to datetime format
excel_path = "\\Users\\User\\OneDrive\\A6\\Excel\\"
excel_path

'\\Users\\User\\OneDrive\\A6\\Excel\\'

In [5]:
file_name = '15-div-23q4.xlsx'
df = pd.read_excel(excel_path + file_name)
df['xdate'] = pd.to_datetime(df['xdate'], errors='coerce')
df['paiddate'] = pd.to_datetime(df['paiddate'], errors='coerce')
df.dtypes

name                object
qtrly              float64
shares               int64
amount               int64
net                float64
xdate       datetime64[ns]
paiddate    datetime64[ns]
cost_amt             int64
pct                float64
actual               int64
dtype: object

In [6]:
df.columns

Index(['name', 'qtrly', 'shares', 'amount', 'net', 'xdate', 'paiddate',
       'cost_amt', 'pct', 'actual'],
      dtype='object')

In [7]:
# Convert column names of df to uppercase for merging with Dividend table
#df.columns = df.columns.str.upper()
df.columns

Index(['name', 'qtrly', 'shares', 'amount', 'net', 'xdate', 'paiddate',
       'cost_amt', 'pct', 'actual'],
      dtype='object')

In [8]:
# Merge the two dataframes based on name column and update XDATE & PAIDDATE columns in Dividend table
merged_df = pd.merge(dividend_table, df[['name', 'xdate', 'paiddate']], left_on='NAME', right_on='name')
merged_df.update(df[['xdate', 'paiddate']], errors='ignore')
dividend_table.update(merged_df)
dividend_table.head()                    

,NAME,Q4,Q3,Q2,Q1,DIVIDEND,PRICE,PERCENT,SHARES,XDATE,PAIDDATE,KIND,ACTUAL
0,WHART,0.1265,0.259,0.193,0.1915,0.7700,0.0,0.0,20000.0,2024-03-08,2024-03-24,,2.0
1,CPNREIT,0.0607,0.257,0.290,0.2470,0.8547,0.0,0.0,60000.0,2024-02-28,2024-03-16,,2.0
2,MCS,0.2000,0.000,0.000,0.0000,0.2000,0.0,0.0,78000.0,2024-03-17,2024-05-08,,2.0
3,AIMIRT,0.2230,0.223,0.223,0.2230,0.8920,0.0,0.0,10000.0,2024-03-15,2024-03-30,,2.0
4,SCC,2.0000,0.000,2.500,0.0000,4.5000,0.0,0.0,600.0,2024-04-07,2024-04-25,,2.0


### mysqldump -u root -p stock dividend > div_backup_file.sql

In [9]:
# Write the updated data back to the database
dividend_table.to_sql('dividend', const, if_exists='replace', index=False)

### mysql -u [username] -p[password] [database_name] < [backup_file.sql]

### From Wizard Coder 13B Q5
### CREATE TABLE dividend_backup AS SELECT * FROM dividend;
### DROP TABLE dividend;
### RENAME TABLE dividend_backup TO dividend;